In [30]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.13.0

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)


In [46]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# Display the version
print(tf.__version__)

# import and mount Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')

2.13.0
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# get path to folder file is in
file_path = '/content/gdrive/MyDrive/apstats-experimentproject'
print(file_path)

/content/gdrive/MyDrive/apstats-experimentproject


In [3]:
# Download the data from tf, unless it's already here.
if not os.path.exists(file_path+'/data/clean'):
  os.mkdir(file_path+'/data/clean')
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
  np.save(file_path+'/data/clean/X_train.npy',X_train)
  np.save(file_path+'/data/clean/y_train.npy',y_train)
  np.save(file_path+'/data/clean/X_test.npy',X_test)
  np.save(file_path+'/data/clean/y_test.npy',y_test)
else:
  X_train = np.load(file_path+'/data/clean/X_train.npy')
  y_train = np.load(file_path+'/data/clean/y_train.npy')
  X_test = np.load(file_path+'/data/clean/X_test.npy')
  y_test = np.load(file_path+'/data/clean/y_test.npy')

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [4]:
# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Decrease pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# Create FGSM Perturbed Data
def create_fgsm_perturbations(data_inputs, data_labels, epsilon = 0.1):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
  with tf.GradientTape() as tape:
    tape.watch(input_tensor)
    output_tensor = model(input_tensor)
    loss = loss_fn(label_tensor, output_tensor)

  gradient = tape.gradient(loss, input_tensor)
  perturbed_input_tensor = input_tensor + epsilon * tf.sign(gradient)
  return tf.cast(perturbed_input_tensor, dtype=input_tensor.dtype)

# Create FGSM perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/FGSM_perturbed'):
  os.mkdir(file_path+'/data/FGSM_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_FGSM_train = np.concatenate(perturbed_data, axis=0)
  X_FGSM_test = create_fgsm_perturbations(X_test, y_test)

  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy',X_FGSM_train)
  np.save(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy',X_FGSM_test)
else:
  X_FGSM_train = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_train.npy')
  X_FGSM_test = np.load(file_path+'/data/FGSM_perturbed/X_FGSM_test.npy')

print("X_FGSM_train shape", X_FGSM_train.shape)
print("y_train shape", y_train.shape)
print("X_FGSM_test shape", X_FGSM_test.shape)
print("y_test shape", y_test.shape)

X_FGSM_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_FGSM_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [6]:
# Clip perturbed training and test data to viable values
X_FGSM_train = np.clip(X_FGSM_train, a_min = 0.0, a_max = 1.0)
X_FGSM_test = np.clip(X_FGSM_test, a_min = 0.0, a_max = 1.0)

In [7]:
# Create PGD Perturbed Data
def create_pgd_perturbations(data_inputs, data_labels, epsilon = 0.01, n_iter = 10):
  input_tensor = tf.convert_to_tensor(data_inputs)
  label_tensor = tf.convert_to_tensor(data_labels)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

  perturbed_tensor = input_tensor
  for i in range(n_iter):
      with tf.GradientTape() as tape:
          tape.watch(perturbed_tensor)
          output_tensor = model(perturbed_tensor)
          loss = loss_fn(label_tensor, output_tensor)

      gradient = tape.gradient(loss, perturbed_tensor)

      perturbed_tensor = perturbed_tensor + gradient
      # Project perturbed_tensor onto the L-infinity ball around input_tensor
      perturbed_tensor = epsilon * tf.sign(
          perturbed_tensor - input_tensor) + input_tensor

  perturbed_tensor = tf.cast(perturbed_tensor, dtype=input_tensor.dtype)
  return perturbed_tensor

# Create PGD perturbations if nonexisting, else load them
if not os.path.exists(file_path+'/data/PGD_perturbed'):
  os.mkdir(file_path+'/data/PGD_perturbed')

  perturbed_data = [None for _ in range(5)]
  for i in range(5):
    perturbed_data[i] = create_fgsm_perturbations(X_train[i * 10000:(i+1) * 10000], y_train[i * 10000:(i+1) * 10000])

  X_PGD_train = np.concatenate(perturbed_data, axis=0)
  X_PGD_test = create_pgd_perturbations(X_test, y_test)

  np.save(file_path+'/data/PGD_perturbed/X_PGD_train.npy',X_PGD_train)
  np.save(file_path+'/data/PGD_perturbed/X_PGD_test.npy',X_PGD_test)
else:
  X_PGD_train = np.load(file_path+'/data/PGD_perturbed/X_PGD_train.npy')
  X_PGD_test = np.load(file_path+'/data/PGD_perturbed/X_PGD_test.npy')

print("X_PGD_train shape", X_PGD_train.shape)
print("y_train shape", y_train.shape)
print("X_PGD_test shape", X_PGD_test.shape)
print("y_test shape", y_test.shape)

X_PGD_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
X_PGD_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [8]:
# Clip perturbed training and test data to viable values
X_PGD_train = np.clip(X_PGD_train, a_min = 0.0, a_max = 1.0)
X_PGD_test = np.clip(X_PGD_test, a_min = 0.0, a_max = 1.0)

In [9]:
from sklearn.model_selection import train_test_split
def createDataSplit(x, y):
  x_first, x_second, y_first, y_second = train_test_split(x, y, test_size=0.5, stratify=y)
  X_clean_model_test, X_FGSM_model_test, y_clean_model_test, y_FGSM_model_test = train_test_split(x_first, y_first, test_size=0.5, stratify=y_first)
  X_PGD_model_test, X_DD_model_test, y_PGD_model_test, y_DD_model_test = train_test_split(x_second, y_second, test_size=0.5, stratify=y_second)
  return X_clean_model_test, y_clean_model_test, X_FGSM_model_test, y_FGSM_model_test, X_PGD_model_test, y_PGD_model_test, X_DD_model_test, y_DD_model_test

In [10]:
X_clean_model_test, y_clean_model_test, X_FGSM_model_test, y_FGSM_model_test, X_PGD_model_test, y_PGD_model_test, X_DD_model_test, y_DD_model_test = createDataSplit(X_PGD_test, y_test)

In [48]:
import pandas as pd

# Load in Clean Model
print("clean model:\n")
model = tf.keras.models.load_model(file_path+'/models/clean.keras')
clean_model_prob = pd.DataFrame(model.predict(X_clean_model_test)[np.arange(2500), y_clean_model_test[:, 0]])
clean_model_prob.to_csv(file_path+'/output_data/clean_model_prob.csv')

# Load in FGSM Model
print("\nfgsm model:\n")
fgsm_model = tf.keras.models.load_model(file_path+'/models/mixed_FGSM.keras')
fgsm_model_prob = pd.DataFrame(fgsm_model.predict(X_FGSM_model_test)[np.arange(2500), y_FGSM_model_test[:, 0]])
fgsm_model_prob.to_csv(file_path+'/output_data/fgsm_model_prob.csv')

# Load in PGD Model
print("\npgd model:\n")
pgd_model = tf.keras.models.load_model(file_path+'/models/mixed_PGD.keras')
pgd_model_prob = pd.DataFrame(pgd_model.predict(X_PGD_model_test)[np.arange(2500), y_PGD_model_test[:, 0]])
pgd_model_prob.to_csv(file_path+'/output_data/pgd_model_prob.csv')

def temperature_cross_entropy(gt, pred):
  loss = tf.nn.softmax_cross_entropy_with_logits(labels=gt, logits=pred/20)
  return loss

# Load in Distilled Model
print("\ndistilled model:\n")
distilled_model = tf.keras.models.load_model(file_path+'/models/distillation_student.keras', custom_objects={'temperature_cross_entropy': temperature_cross_entropy})
dd_model_prob = pd.DataFrame(tf.nn.softmax(distilled_model.predict(X_DD_model_test)).numpy()[np.arange(2500), y_DD_model_test[:, 0]])
dd_model_prob.to_csv(file_path+'/output_data/dd_model_prob.csv')

clean model:

79/79 [==============================] - 3s 42ms/step

fgsm model:

79/79 [==============================] - 5s 55ms/step

pgd model:

79/79 [==============================] - 3s 31ms/step

distilled model:

79/79 [==============================] - 3s 31ms/step
